<a href="https://colab.research.google.com/github/egorostap/other_projects/blob/main/fb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install facebookads==2.5.0
from facebookads.api import FacebookAdsApi
!pip install facebook_business
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccountuser import AdAccountUser as AdUser
from facebook_business import adobjects
from matplotlib import pyplot as plt
import time
from facebook_business.api import FacebookAdsApi
from pandas import DataFrame

Написание скрипта
Создадим три переменных, в которые запишем access token, App ID и App Secret. Авторизуемся через метод init() класса FacebooksAdsApi и добавляем пользователя. Метод get_ad_accounts() вернёт нам данные по всем нашим рекламным аккаунтам в виде словаря. По этим же данным можем получить информацию о кампаниях методом get_campaigns().

In [ ]:
my_access_token = 'EAAwDjsTCk8kBALCiX8raWRmLQeezOwkLIWNZAFGTHAgZCqdWsKkNZCGiDouJhODJ5m25PI79z65uvL6wVPRX4rhFtQ0RIS7gY0ZCh693DJQh6SSpa7d7XCtfZAn9rd6ZB4RW4AsRCk1non2gsS20gzQYGDDaHG4AUbs6s3qCxt2T6RgJZAnUqURDuDN376uHJIW50zZCR6pbRkh6xtiTULAJBZAvM46434z8ZD'
my_app_id = '3381611441853385'
my_app_secret = 'a0d38d2d8eba2fa13ff2d7246f344dd1'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

# me = AdUser(fbid='me')
# my_accounts = list(me.get_ad_accounts())
# my_accounts

my_account = AdAccount('act_426823688657953')
my_accounts = [my_account]
campaigns = my_account.get_campaigns()
print(campaigns)

# my_account
# my_account = my_accounts[0]


Попробуем получить amount spent через my_account. Для этого воспользуемся методом api_get(), передав в параметр fields поле AdAccount.Field.amount_spent. Теперь, чтобы получить желаемые данные, выведем поле у переменной my_account, поделив на 100, чтобы обрубить копейки. Расходы получаем в валюте аккаунта, в нашем случае это RUB. То, ради чего мы всё это затеваем — получить данные о расходах на рекламные кампании для последующего анализа.

In [4]:
my_account.api_get(fields=[AdAccount.Field.amount_spent])
print(int(my_account[AdAccount.Field.amount_spent])/100)

435503.26


Объявим переменную fields — в этом списке будут храниться поля, которые мы хотим получать: id кампании, количество кликов, затрат и просмотров. Теперь опишем две функции. Первая будет асинхронно отправлять запросы к Facebook и возвращать результаты. Вторая — формирует эти запросы и передает в первую функцию. В результате будем получать список словарей.

In [15]:
fields = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.campaign_name,
    AdsInsights.Field.clicks,
    AdsInsights.Field.spend,
    AdsInsights.Field.impressions,
    AdsInsights.Field.actions
]

count = 0

def wait_for_async_job(async_job):
    global count
    async_job = async_job.api_get()
    while async_job[AdReportRun.Field.async_status] != 'Job Completed' or async_job[
        AdReportRun.Field.async_percent_completion] < 100:
        time.sleep(2)
        async_job = async_job.api_get()
    else:
        print("Job " + str(count) + " completed")
        count += 1
    return async_job.get_result(params={"limit": 1000})

a = [{'field':"action_type",'operator':"IN",'value':["lead"]}]

def get_insights(account, date_preset='last_7d'):
    account = AdAccount(account["id"])
    i_async_job = account.get_insights(
        params={
            'level': 'ad',
            'date_preset': date_preset,
            'time_increment': 1,
            'filtering' : a
            },
            fields=fields,
            is_async=True)

    results = [dict(item) for item in wait_for_async_job(i_async_job)]
    return results
# get_insights(my_account, date_preset='last_7d')

Следующий шаг — получение искомых данных о затратах. Будем собирать данные за всё время, поэтому заведём переменную date_preset, значение которой поставим lifetime. И для каждого аккаунта вызовем функцию get_insights(), а список, который она возвращает, положим в insights_lists.
Создадим DataFrame и вытащим из insights_lists интересующие данные — это id кампании, количество кликов, затраты и просмотры.

In [27]:


elem_insights = []
insights_lists = []
date_preset = 'last_90d'
for elem in my_accounts:
            elem_insights = get_insights(elem, date_preset)
            insights_lists.append(elem_insights)

insight_campaign_id_list = []
insight_clicks_list = []
insight_spend_list = []
insight_impressions_list = []
insight_date_start_list = []
insight_date_stop_list = []
insight_leads_list = []
insight_campaign_name = []
for elem1 in insights_lists:
    for elem2 in elem1:
        insight_campaign_id_list.append(int(elem2['campaign_id']))
        insight_clicks_list.append(int(elem2['clicks']))
        insight_spend_list.append(float(elem2['spend']))
        insight_impressions_list.append(int(elem2['impressions']))
        insight_date_start_list.append(elem2['date_start'])
        insight_date_stop_list.append(elem2['date_stop'])
        insight_campaign_name.append(elem2['campaign_name'])
        insight_leads_list.append(elem2['actions'][0]['value'] if 'actions' in elem2 else 0)

        # print(elem2['actions'])


df = DataFrame()

df['campaign_name'] = insight_campaign_name
df['campaign_id'] = insight_campaign_id_list
df['spend'] = insight_spend_list
df['leads'] = insight_leads_list
df['clicks'] = insight_clicks_list
df['impressions'] = insight_impressions_list
df['date_start'] = insight_date_start_list
df['date_stop'] = insight_date_stop_list
df.groupby(['campaign_name']).sum()


df.head(100)

Job 2 completed


,campaign_name,campaign_id,spend,leads,clicks,impressions,date_start,date_stop
0,skandinavia-qwz-fb-inst,23846900473100216,3.91,0,2,19,2021-08-13,2021-08-13
1,skandinavia-qwz-fb-inst,23846900473100216,225.77,0,63,2054,2021-08-13,2021-08-13
2,familypark-qwz-fb-inst,23846885675190216,384.34,1,63,3725,2021-08-13,2021-08-13
3,familypark-qwz-fb-inst,23846885675190216,129.56,2,30,1628,2021-08-13,2021-08-13
4,skandinavia-qwz-fb-inst,23846900473100216,17.08,0,3,105,2021-08-13,2021-08-13
...,...,...,...,...,...,...,...,...
95,skandinavia-qwz-fb-inst,23846900473100216,342.16,0,37,2071,2021-08-18,2021-08-18
96,lugovoe,23848205134100216,7.41,0,0,33,2021-08-18,2021-08-18
97,lugovoe,23848205134100216,321.44,0,37,1669,2021-08-18,2021-08-18
98,lugovoe,23848205134100216,2.05,0,0,7,2021-08-18,2021-08-18


In [20]:
# import pandas as pd
!pip install gspread==3.6.0
import gspread
from gspread_dataframe import set_with_dataframe

In [28]:
# ACCES GOOGLE SHEET
sa = gspread.service_account(filename="/usr/local/lib/python3.7/dist-packages/gspread/service_account.json")
sh = sa.open_by_key('1P3vTJeEnowD_Q1z9UO32hdI-tofzVcdi6fEeAKGdUKw')
worksheet = sh.worksheet("zag_dom")

# CLEAR SHEET CONTENT
range_of_cells = worksheet.range('A2:H1000') #-> Select the range you want to clear
for cell in range_of_cells:
    cell.value = ''
worksheet.update_cells(range_of_cells) 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, df) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET


# sh = sa.open("Статистика кампании FB Егор")
# gc = gspread.service_account(filename='client_secret_1038106429640-f9ff5mosk926988bq7stb03sleg22863.apps.googleusercontent.com.json')
# gc = gspread.authorze(filename='client_secret_1038106429640-f9ff5mosk926988bq7stb03sleg22863.apps.googleusercontent.com.json')
# worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# worksheet.update('A1:B2', [[1, 2], [3, 4]])



# your_dataframe = pd.df()



In [6]:
!python --version


Python 3.7.12
